In [5]:
library(Seurat)
library(ggplot2)
library(CellChat)
library(BiocParallel)
setwd("/data/TAK_981")

In [6]:
adata <-schard::h5ad2seurat('h5ad/batch_corrected.h5ad', use.raw = TRUE)


In [7]:
Idents(adata) <- "annot"
adata <- subset(adata, idents = c("Acinar", "B Cells", "RBC"), invert=TRUE)

In [8]:
cond<-SplitObject(adata, split.by = "condition")

In [9]:
runCellChat <- function(subset_ad){
    assay <- GetAssayData(subset_ad, assay = "RNA", slot = "counts")
    assay <- normalizeData(assay)
    Idents(subset_ad) <- "annot"
    labels <- Idents(subset_ad)
    meta <- subset_ad[[]]
    meta$samples <- meta$sample_id
    meta$labels <- meta$annot
    rm(subset_ad)
    cellchat <- createCellChat(object = assay, group.by = "annot", meta=meta)
    CellChatDB <- CellChatDB.mouse
    CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling")
    cellchat@DB <- CellChatDB.use
    cellchat <- subsetData(cellchat)
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat)
    cellchat <- computeCommunProb(cellchat, type = "triMean", trim = NULL, raw.use = TRUE)
    cellchat <- filterCommunication(cellchat, min.cells = 10)
    cellchat <- computeCommunProbPathway(cellchat)
    cellchat <- aggregateNet(cellchat)
    cellchat
}

In [10]:
cellchat_objs <- bplapply(cond, FUN=runCellChat, BPPARAM = MulticoreParam(2))


R_zmq_msg_send errno: 4 strerror: Interrupted system call
[1] "Create a CellChat object from a data matrix"
R_zmq_msg_send errno: 4 strerror: Interrupted system call
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  CD103+ cDC, Epithelial, Fibroblasts, Granulocytes, Macrophages, NK Cells, T-Cells 
The number of highly variable ligand-receptor pairs used for signaling inference is 817 
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2024-11-13 00:14:12]"
  |======================================================================| 100%
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2024-11-13 00:17:35]"
[1] "Create a CellChat object from a data matrix"
R_zmq_msg_send errno: 4 strerror: Interrupted system call
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  CD103+ cDC,

In [11]:
saveRDS(file = "h5ad/cellchat/individ_cellchat.RData", object = cellchat_objs)